In [10]:
// prio_hol_open example - multiclass with PS, SIRO, FCFS, HOL priority
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.solvers.ssa.*

In [11]:
// Create network
val model = Network("MyNetwork")

// Block 1: nodes
val node1 = Source(model, "Source")
val node2 = Queue(model, "Queue1", SchedStrategy.FCFS)
val node3 = Queue(model, "Queue2", SchedStrategy.SIRO)
val node4 = Queue(model, "Queue3", SchedStrategy.PS)
val node5 = Queue(model, "Queue4", SchedStrategy.HOL)
val node6 = Sink(model, "Sink")

In [12]:
// Block 2: classes
val jobclass1 = OpenClass(model, "Class1", 0)
val jobclass2 = OpenClass(model, "Class2", 1) // higher priority
val jobclass3 = OpenClass(model, "Class3", 0)

In [13]:
// Set arrival rates
node1.setArrival(jobclass1, Exp.fitMean(10.0))
node1.setArrival(jobclass2, Exp.fitMean(10.0))
node1.setArrival(jobclass3, Exp.fitMean(10.0))

In [14]:
// Set service times for all nodes and classes
node2.setService(jobclass1, Exp.fitMean(0.3)) // WebServer, Class0
node2.setService(jobclass2, Exp.fitMean(0.5)) // WebServer, Class1
node2.setService(jobclass3, Exp.fitMean(0.6)) // WebServer, Class2

node3.setService(jobclass1, Exp.fitMean(1.1)) // Storage1, Class0
node3.setService(jobclass2, Exp.fitMean(1.3)) // Storage1, Class1
node3.setService(jobclass3, Exp.fitMean(1.5)) // Storage1, Class2

node4.setService(jobclass1, Exp.fitMean(2.0)) // Storage2, Class0
node4.setService(jobclass2, Exp.fitMean(2.1)) // Storage2, Class1
node4.setService(jobclass3, Exp.fitMean(1.9)) // Storage2, Class2

node5.setService(jobclass1, Exp.fitMean(2.5)) // Storage3, Class0
node5.setService(jobclass2, Exp.fitMean(1.9)) // Storage3, Class1
node5.setService(jobclass3, Exp.fitMean(4.3)) // Storage3, Class2

In [15]:
// Block 3: topology - initialize routing matrix
val P = model.initRoutingMatrix()

// Class1 routing
P.set(jobclass1, jobclass1, node1, node2, 1.0) // Source -> WebServer
P.set(jobclass1, jobclass1, node2, node3, 0.25) // WebServer -> Storage1
P.set(jobclass1, jobclass1, node2, node4, 0.25) // WebServer -> Storage2
P.set(jobclass1, jobclass1, node2, node5, 0.25) // WebServer -> Storage3
P.set(jobclass1, jobclass1, node2, node6, 0.25) // WebServer -> Sink
P.set(jobclass1, jobclass1, node3, node2, 1.0) // Storage1 -> WebServer
P.set(jobclass1, jobclass1, node4, node2, 1.0) // Storage2 -> WebServer
P.set(jobclass1, jobclass1, node5, node2, 1.0) // Storage3 -> WebServer

// Class2 routing
P.set(jobclass2, jobclass2, node1, node2, 1.0) // Source -> WebServer
P.set(jobclass2, jobclass2, node2, node3, 0.25) // WebServer -> Storage1
P.set(jobclass2, jobclass2, node2, node4, 0.25) // WebServer -> Storage2
P.set(jobclass2, jobclass2, node2, node5, 0.25) // WebServer -> Storage3
P.set(jobclass2, jobclass2, node2, node6, 0.25) // WebServer -> Sink
P.set(jobclass2, jobclass2, node3, node2, 1.0) // Storage1 -> WebServer
P.set(jobclass2, jobclass2, node4, node2, 1.0) // Storage2 -> WebServer
P.set(jobclass2, jobclass2, node5, node2, 1.0) // Storage3 -> WebServer

// Class3 routing
P.set(jobclass3, jobclass3, node1, node2, 1.0) // Source -> WebServer
P.set(jobclass3, jobclass3, node2, node3, 0.25) // WebServer -> Storage1
P.set(jobclass3, jobclass3, node2, node4, 0.25) // WebServer -> Storage2
P.set(jobclass3, jobclass3, node2, node5, 0.25) // WebServer -> Storage3
P.set(jobclass3, jobclass3, node2, node6, 0.25) // WebServer -> Sink
P.set(jobclass3, jobclass3, node3, node2, 1.0) // Storage1 -> WebServer
P.set(jobclass3, jobclass3, node4, node2, 1.0) // Storage2 -> WebServer
P.set(jobclass3, jobclass3, node5, node2, 1.0) // Storage3 -> WebServer

model.link(P)

In [18]:
// Solve with different solvers
CTMC(model, "seed", 23000, "cutoff", 1).avgTable.print()
MVA(model, "seed", 23000).avgTable.print()
JMT(model, "seed", 23000, "samples", 10000).avgTable.print()
SSA(model, "seed", 23000, "samples", 10000).avgTable.print()

java.lang.RuntimeException: Unable to compute results and therefore unable to print AvgTable.